In [1]:
!pip install openai==1.13.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [1]:

import asyncio
from openai import AsyncAzureOpenAI
import requests


In [2]:
# Configuration settings
azure_oai_endpoint ="https://eyuser20.openai.azure.com/"
azure_oai_key ="1379196e9ab94ff6995d28aa0a9ca9f2"
azure_oai_deployment ="GPT-35"

In [3]:
# Configure the Azure OpenAI client
client = AsyncAzureOpenAI(
    azure_endpoint=azure_oai_endpoint,
    api_key=azure_oai_key,
    api_version="2024-02-15-preview"
)


print("Azure OpenAI client configured.")

Azure OpenAI client configured.


In [8]:
# Set to True to print the full response from OpenAI for each call
printFullResponse = False

async def main():
    try:
        while True:
            system_text = "You are a helpful medical bot with expertise in medical field."
            print("Enter 1 to chat 2 to generate image 3 to exit")
            choice = int(input("Enter your choice: "))
            if(choice == 1):
                user_text = input("Enter user message: ")
                await call_openai_model(system_message=system_text,
                                        user_message=user_text,
                                        model=azure_oai_deployment,
                                        client=client)
            elif(choice == 2):
                prompt = input("Enter image prompt: ")
                image_url = await call_dalle_model(prompt)
                print(image_url)
            else:
                print('Exiting program...')
                break

    except Exception as ex:
        print(ex)

In [12]:
async def call_openai_model(system_message, user_message, model, client):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "What should I do if I have toothache"},
        {"role": "assistant", "content": "When you have a tootache, vist a dentist and get your teeth checked. Brush your teeth regurlary and twice a day"},
        {"role": "user", "content": "What kind of exercises can i do for alleviating back pain?"},
        {"role": "assistant", "content": "To alleviate the back pain you should regulary do exercises for both the upper and lower back. You can do planks, crunches, leg raises and deadlift"},
        {"role": "user", "content": user_message},
        ]
    print("Sending request to Azure OpenAI model...")
    # Call the Azure OpenAI model
    response = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=800
    )

    if printFullResponse:
        print(response)

    print("Response:" + response.choices[0].message.content + "\n")

In [16]:
async def call_dalle_model(prompt):
    dalle_azure_oai_endpoint ="https://eyuser3.openai.azure.com/"
    dalle_azure_oai_key ="06f6ae63b2f94f23b2c39ec67a612ee8"
    dalle_azure_oai_deployment ="Dalle3"
    api_version="2024-02-15-preview"
    url = "{}openai/deployments/dalle3/images/generations?api-version={}".format(dalle_azure_oai_endpoint, api_version)
    headers= { "api-key": dalle_azure_oai_key, "Content-Type": "application/json" }
    body = {
                "prompt": prompt,
                "n": 1,
                "size": "1024x1024"
            }
    response = requests.post(url, headers=headers, json=body)
    print(response.text)
      # Get the revised prompt and image URL from the response
    revised_prompt = response.json()['data'][0]['revised_prompt']
    image_url = response.json()['data'][0]['url']

    # Display the URL for the generated image
    print(revised_prompt)
    print(image_url)
    return image_url

In [17]:
await main()

Enter 1 to chat 2 to generate image 3 to exit
Enter your choice: 2
Enter image prompt: x-ray image of human chest
{"created":1718192587,"data":[{"content_filter_results":{"hate":{"filtered":false,"severity":"safe"},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}},"prompt_filter_results":{"hate":{"filtered":false,"severity":"safe"},"profanity":{"detected":false,"filtered":false},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}},"revised_prompt":"An x-ray image showcasing the human chest. The radiography must consist of the sternum (breastbone) in the center, the ribcage surrounding it, the clavicle (collarbone) at the top, and the heart and lungs visible as darker areas. The lung fields must look symmetrical on each side. The faint white branching structures might be the bronchial tree. The image cont